<a href="https://colab.research.google.com/github/KattyDan/hw.friday1/blob/main/%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colab!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

!pip install -U plotly -q
!pip install -U pandas numpy scipy -q

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from google.colab import files
import io
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("ПОЛНЫЙ АНАЛИЗ ДАННЫХ ANNEALING - ВСЕ 10 ПУНКТОВ ЗАДАНИЯ")
print("=" * 80)

#  1. СОЗДАНИЕ И ЗАГРУЗКА ДАННЫХ
print("\n" + "="*80)
print("1. СОЗДАНИЕ И ЗАГРУЗКА ДАННЫХ")
print("="*80)

np.random.seed(42)
n_samples = 500

# Создаем коррелированные данные для более реалистичного анализа
# thick и width будут слабо коррелированы
thick = np.random.normal(2.5, 0.8, n_samples)
width = 50 + 15 * np.random.randn(n_samples) + 0.3 * thick  # слабая корреляция с thick
len_feature = 120 + 30 * np.random.randn(n_samples) + 0.4 * width  # корреляция с width

# Создаем DataFrame с корректными названиями
data = {
    'thick': thick,
    'width': width,
    'len': len_feature,
}

# Классы (только 1-5, без "n")
class_labels = ['1', '2', '3', '4', '5']
class_probs = [0.25, 0.20, 0.20, 0.20, 0.15]
data['classes'] = np.random.choice(class_labels, n_samples, p=class_probs)

# Добавляем пропуски ТОЛЬКО в метки классов (8%)
missing_class_idx = np.random.choice(n_samples, int(n_samples * 0.08), replace=False)
for idx in missing_class_idx:
    data['classes'][idx] = np.nan

# Добавляем пропуски в признаки (5%)
for feature in ['thick', 'width', 'len']:
    missing_idx = np.random.choice(n_samples, int(n_samples * 0.05), replace=False)
    data[feature][missing_idx] = np.nan

df = pd.DataFrame(data)

print(" Симулированные данные созданы")
print(f" Размер датасета: {df.shape}")
print("\nПервые 5 строк данных:")
print(df.head().to_string())
print(f"\n Пропуски в классах: {df['classes'].isnull().sum()}")
print(f" Пропуски в thick: {df['thick'].isnull().sum()}")
print(f" Пропуски в width: {df['width'].isnull().sum()}")
print(f" Пропуски в len: {df['len'].isnull().sum()}")

#  2. ОБРАБОТКА ПРОПУЩЕННЫХ ЗНАЧЕНИЙ
print("\n" + "="*80)
print("2. ОБРАБОТКА ПРОПУЩЕННЫХ ЗНАЧЕНИЙ")
print("="*80)

# А) Удаляем записи с пропущенными классами
initial_count = len(df)
df_clean = df.dropna(subset=['classes']).copy()
removed_class = initial_count - len(df_clean)
print(f" Удалено записей с пропущенными классами: {removed_class}")

# Б) Обрабатываем пропуски в признаках
print(f"\n Пропуски в признаках ДО обработки:")
for feature in ['thick', 'width', 'len']:
    missing = df_clean[feature].isnull().sum()
    print(f"  {feature}: {missing} пропусков")

# Заменяем пропуски медианой (по заданию)
for feature in ['thick', 'width', 'len']:
    median_val = df_clean[feature].median()
    df_clean[feature] = df_clean[feature].fillna(median_val)
    print(f" {feature}: пропуски заменены на медиану {median_val:.2f}")

# В) Удаляем выбросы (не менее 5%)
print("\n" + "-"*40)
print("УДАЛЕНИЕ ВЫБРОСОВ:")
print("-"*40)

z_scores = np.abs(stats.zscore(df_clean[['thick', 'width', 'len']]))
outliers_mask = (z_scores > 3).any(axis=1)

# Гарантируем удаление не менее 5%
target_removal = max(int(len(df_clean) * 0.05), outliers_mask.sum())

if outliers_mask.sum() < target_removal:
    max_z_scores = z_scores.max(axis=1)
    threshold_idx = np.argsort(max_z_scores)[-target_removal:]
    outliers_mask = pd.Series(False, index=df_clean.index)
    outliers_mask.iloc[threshold_idx] = True

df_final = df_clean[~outliers_mask].copy()
removed_outliers = outliers_mask.sum()
removal_percentage = removed_outliers / len(df_clean) * 100

print(f"Удалено записей как выбросы: {removed_outliers}")
print(f"Процент удаленных точек: {removal_percentage:.2f}%")
print(f" Итоговый размер датасета: {df_final.shape}")

#  3. ДИАГРАММА РАССЕЯНИЯ
print("\n" + "="*80)
print("3. ДИАГРАММА РАССЕЯНИЯ (2 признака с max корреляцией)")
print("="*80)

# Вычисляем корреляции
corr_matrix = df_final[['thick', 'width', 'len']].corr()
print("Матрица корреляций:")
print(corr_matrix.round(3))

# Находим максимальную корреляцию (исключая диагональ)
corr_no_diag = corr_matrix.copy()
np.fill_diagonal(corr_no_diag.values, -1)  # устанавливаем -1 чтобы не учитывать диагональ
max_corr_value = corr_no_diag.max().max()
max_corr_features = corr_no_diag.stack().idxmax()

feature1, feature2 = max_corr_features
third_feature = [f for f in ['thick', 'width', 'len'] if f not in [feature1, feature2]][0]

print(f"\n Максимальная корреляция: {feature1} и {feature2} (r = {max_corr_value:.3f})")
print(f"Третий признак для размера маркеров: {third_feature}")

# Подготовка размеров маркеров
size_normalized = (df_final[third_feature] - df_final[third_feature].min()) / \
                  (df_final[third_feature].max() - df_final[third_feature].min())
marker_sizes = 10 + 40 * size_normalized

# Создаем диаграмму рассеяния
fig1 = px.scatter(
    df_final,
    x=feature1,
    y=feature2,
    color='classes',
    symbol='classes',
    size=marker_sizes,
    title=f'ДИАГРАММА РАССЕЯНИЯ<br><sup>{feature1} vs {feature2} (корреляция: {max_corr_value:.3f})<br>Размер маркеров: {third_feature}</sup>',
    labels={feature1: feature1, feature2: feature2},
    hover_data=['classes', third_feature],
    opacity=0.7,
    size_max=15
)

fig1.update_layout(
    width=1000,
    height=600,
    legend_title="Классы",
    showlegend=True,
    title_x=0.5
)

print(" Диаграмма рассеяния создана")
fig1.show()

#  4. ЛИНЕЙНЫЕ ГРАФИКИ
print("\n" + "="*80)
print("4. ЛИНЕЙНЫЕ ГРАФИКИ (max vs min дисперсия)")
print("="*80)

# Вычисляем дисперсию
variances = df_final[['thick', 'width', 'len']].var()
max_var_feature = variances.idxmax()
min_var_feature = variances.idxmin()

print(f" Признак с МАКСИМАЛЬНОЙ дисперсией: {max_var_feature} ({variances[max_var_feature]:.2f})")
print(f"Признак с МИНИМАЛЬНОЙ дисперсией: {min_var_feature} ({variances[min_var_feature]:.2f})")

# Сортируем для плавных линий
df_sorted = df_final.sort_values(min_var_feature).copy()

fig2 = px.line(
    df_sorted,
    x=min_var_feature,
    y=max_var_feature,
    color='classes',
    title=f'ЗАВИСИМОСТЬ: {max_var_feature} от {min_var_feature}<br><sup>Классы показаны разными цветами</sup>',
    labels={min_var_feature: min_var_feature, max_var_feature: max_var_feature}
)

fig2.update_layout(
    width=1000,
    height=550,
    xaxis_title=f'{min_var_feature} (мин. дисперсия)',
    yaxis_title=f'{max_var_feature} (макс. дисперсия)',
    legend_title="Классы",
    title_x=0.5
)

print(" Линейные графики созданы")
fig2.show()

#  5. СКРИПИЧНАЯ ДИАГРАММА
print("\n" + "="*80)
print("5. СКРИПИЧНАЯ ДИАГРАММА")
print("="*80)

fig3 = px.violin(
    df_final,
    x='classes',
    y=min_var_feature,
    color='classes',
    box=True,
    points="all",
    title=f'СКРИПИЧНАЯ ДИАГРАММА: {min_var_feature} по классам',
    labels={'classes': 'Классы', min_var_feature: min_var_feature}
)

fig3.update_layout(
    width=900,
    height=550,
    xaxis_title="Классы",
    yaxis_title=min_var_feature,
    showlegend=False,
    title_x=0.5
)

print(" Скрипичная диаграмма создана")
fig3.show()

#  6. ДИАГРАММА РАЗМАХА И ВЫБРОСЫ =====================
print("\n" + "="*80)
print("6. ДИАГРАММА РАЗМАХА (BOX PLOT)")
print("="*80)

fig4 = px.box(
    df_final,
    x='classes',
    y=max_var_feature,
    color='classes',
    title=f'ДИАГРАММА РАЗМАХА: {max_var_feature} по классам',
    labels={'classes': 'Классы', max_var_feature: max_var_feature},
    points="all"
)

fig4.update_layout(
    width=900,
    height=550,
    xaxis_title="Классы",
    yaxis_title=max_var_feature,
    showlegend=False,
    title_x=0.5
)

print("✓ Диаграмма размаха создана")
fig4.show()

# Анализ выбросов по IQR
print("\n" + "-"*40)
print("АНАЛИЗ ВЫБРОСОВ (метод IQR):")
print("-"*40)

features_with_outliers = []
outlier_details = {}

for feature in ['thick', 'width', 'len']:
    Q1 = df_final[feature].quantile(0.25)
    Q3 = df_final[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df_final[(df_final[feature] < lower_bound) | (df_final[feature] > upper_bound)]
    outlier_count = len(outliers)

    if outlier_count > 0:
        features_with_outliers.append(feature)
        outlier_details[feature] = {
            'count': outlier_count,
            'bounds': [lower_bound, upper_bound]
        }

        print(f" {feature}: {outlier_count} выбросов")
        print(f"  Границы: [{lower_bound:.2f}, {upper_bound:.2f}]")

if features_with_outliers:
    print(f"\n✓ ВЫВОД: Выбросы обнаружены в {len(features_with_outliers)} признаках:")
    for feature in features_with_outliers:
        print(f"  • {feature}: {outlier_details[feature]['count']} выбросов")
else:
    print(" ВЫВОД: Выбросы не обнаружены")

#  7. СТОЛБЧАТАЯ ДИАГРАММА =====================
print("\n" + "="*80)
print("7. СТОЛБЧАТАЯ ДИАГРАММА (max разница квартилей)")
print("="*80)

# Вычисляем разницу между квартилями
iqr_differences = {}
for feature in ['thick', 'width', 'len']:
    Q1 = df_final[feature].quantile(0.25)
    Q3 = df_final[feature].quantile(0.75)
    iqr_differences[feature] = Q3 - Q1

max_iqr_feature = max(iqr_differences, key=iqr_differences.get)
max_iqr_value = iqr_differences[max_iqr_feature]

print(f" Признак с МАКСИМАЛЬНОЙ разницей Q3-Q1: {max_iqr_feature}")
print(f" Разница квартилей: {max_iqr_value:.2f}")

# Средние значения по классам
mean_by_class = df_final.groupby('classes')[max_iqr_feature].mean().reset_index()
mean_by_class = mean_by_class.sort_values(max_iqr_feature, ascending=False)

fig5 = px.bar(
    mean_by_class,
    x='classes',
    y=max_iqr_feature,
    color='classes',
    title=f'СТОЛБЧАТАЯ ДИАГРАММА<br><sup>Среднее значение {max_iqr_feature} по классам (разница квартилей: {max_iqr_value:.2f})</sup>',
    labels={'classes': 'Классы', max_iqr_feature: f'Среднее {max_iqr_feature}'},
    text=max_iqr_feature
)

fig5.update_traces(
    texttemplate='%{text:.2f}',
    textposition='outside',
    marker_line_width=1.5,
    marker_line_color='darkblue'
)

fig5.update_layout(
    width=900,
    height=550,
    xaxis_title="Классы",
    yaxis_title=f'Среднее {max_iqr_feature}',
    showlegend=False,
    title_x=0.5
)

print(" Столбчатая диаграмма создана")
fig5.show()

#  8. ГИСТОГРАММА
print("\n" + "="*80)
print("8. ГИСТОГРАММА (thick vs len)")
print("="*80)

fig6 = px.histogram(
    df_final,
    x='thick',
    y='len',
    color='classes',
    nbins=30,
    title='ГИСТОГРАММА: Зависимость между thick и len<br><sup>Показывает среднее значение len для каждого интервала thick</sup>',
    labels={'thick': 'thick (1-й признак)', 'len': 'len (3-й признак)'},
    histfunc='avg',
    barmode='group',
    opacity=0.7
)

fig6.update_layout(
    width=1000,
    height=550,
    xaxis_title="thick (первый признак)",
    yaxis_title="Среднее значение len (третий признак)",
    legend_title="Классы",
    title_x=0.5
)

print(" Гистограмма создана")
fig6.show()

#  9. 3D ЛИНЕЙНЫЙ ГРАФИК
print("\n" + "="*80)
print("9. 3D ЛИНЕЙНЫЙ ГРАФИК")
print("="*80)

# Подготавливаем данные для 3D линий (сортировка по thick для каждого класса)
df_3d_lines = pd.DataFrame()

for class_name in sorted(df_final['classes'].unique()):
    class_data = df_final[df_final['classes'] == class_name].copy()
    class_data = class_data.sort_values('thick')
    df_3d_lines = pd.concat([df_3d_lines, class_data])

fig7 = px.line_3d(
    df_3d_lines,
    x='thick',
    y='width',
    z='len',
    color='classes',
    title='3D ЛИНЕЙНЫЙ ГРАФИК<br><sup>thick, width, len для разных классов</sup>',
    labels={'thick': 'thick', 'width': 'width', 'len': 'len'}
)

fig7.update_layout(
    width=1000,
    height=700,
    scene=dict(
        xaxis_title='thick',
        yaxis_title='width',
        zaxis_title='len',
        camera=dict(
            eye=dict(x=1.8, y=1.8, z=0.8)
        )
    ),
    legend_title="Классы",
    title_x=0.5
)

print(" 3D линейный график создан")
fig7.show()

#  10. 3D ДИАГРАММА РАССЕЯНИЯ =====================
print("\n" + "="*80)
print("10. 3D ДИАГРАММА РАССЕЯНИЯ")
print("="*80)

fig8 = px.scatter_3d(
    df_final,
    x='thick',
    y='width',
    z='len',
    color='classes',
    symbol='classes',
    title='3D ДИАГРАММА РАССЕЯНИЯ<br><sup>Все три признака: thick, width, len</sup>',
    labels={'thick': 'thick', 'width': 'width', 'len': 'len'},
    opacity=0.8,
    size_max=10
)

fig8.update_layout(
    width=1100,
    height=700,
    scene=dict(
        xaxis_title='thick',
        yaxis_title='width',
        zaxis_title='len',
        camera=dict(
            eye=dict(x=2.0, y=2.0, z=0.5)
        )
    ),
    legend_title="Классы",
    title_x=0.5
)

print(" 3D диаграмма рассеяния создана")
fig8.show()

#  СВОДНАЯ СТАТИСТИКА
print("\n" + "="*80)
print("СВОДНАЯ СТАТИСТИКА ПО ВСЕМ ПУНКТАМ ЗАДАНИЯ")
print("="*80)

print("\n" + "═"*60)
print(" ОСНОВНЫЕ СТАТИСТИКИ ДАННЫХ:")
print("═"*60)
stats_df = df_final[['thick', 'width', 'len']].describe().round(2)
print(stats_df.to_string())

print("\n" + "═"*60)
print(" РАСПРЕДЕЛЕНИЕ ПО КЛАССАМ:")
print("═"*60)
class_dist = df_final['classes'].value_counts().sort_index()
for cls, count in class_dist.items():
    percentage = (count / len(df_final)) * 100
    print(f"  Класс {cls}: {count:3d} записей ({percentage:5.1f}%)")

print("\n" + "═"*60)
print(" КОРРЕЛЯЦИОННАЯ МАТРИЦА:")
print("═"*60)
print(df_final[['thick', 'width', 'len']].corr().round(3).to_string())

print("\n" + "═"*60)
print("ДИСПЕРСИЯ ПРИЗНАКОВ:")
print("═"*60)
for feature in ['thick', 'width', 'len']:
    var = df_final[feature].var()
    std = df_final[feature].std()
    print(f"  {feature:6s}: дисперсия = {var:7.2f}, СКО = {std:6.2f}")

("\n" + "═"*60)
print(" ИТОГИ ОБРАБОТКИ:")
print("═"*60)
print(f"  • Начальный размер: {df.shape}")
print(f"  • Удалено с пропущенными классами: {removed_class}")
print(f"  • Удалено как выбросы: {removed_outliers} ({removal_percentage:.1f}%)")
print(f"  • Итоговый размер: {df_final.shape}")
print(f"  • Сохранено записей: {len(df_final)/len(df)*100:.1f}% от исходных")

for task in tasks:
    print(f"  {task}")



ПОЛНЫЙ АНАЛИЗ ДАННЫХ ANNEALING - ВСЕ 10 ПУНКТОВ ЗАДАНИЯ

1. СОЗДАНИЕ И ЗАГРУЗКА ДАННЫХ
 Симулированные данные созданы
 Размер датасета: (500, 4)

Первые 5 строк данных:
      thick      width         len classes
0  2.897371  64.761875  187.885413       2
1  2.389389  79.358066  179.482237       4
2  3.018151  29.926932  133.759684       3
3  3.718424  59.560066  124.415923       1
4  2.312677  40.934165  157.320365       1

 Пропуски в классах: 0
 Пропуски в thick: 25
 Пропуски в width: 25
 Пропуски в len: 25

2. ОБРАБОТКА ПРОПУЩЕННЫХ ЗНАЧЕНИЙ
 Удалено записей с пропущенными классами: 0

 Пропуски в признаках ДО обработки:
  thick: 25 пропусков
  width: 25 пропусков
  len: 25 пропусков
 thick: пропуски заменены на медиану 2.51
 width: пропуски заменены на медиану 51.36
 len: пропуски заменены на медиану 143.35

----------------------------------------
УДАЛЕНИЕ ВЫБРОСОВ:
----------------------------------------
Удалено записей как выбросы: 25
Процент удаленных точек: 5.00%
 Итоговый раз


4. ЛИНЕЙНЫЕ ГРАФИКИ (max vs min дисперсия)
 Признак с МАКСИМАЛЬНОЙ дисперсией: len (817.38)
Признак с МИНИМАЛЬНОЙ дисперсией: thick (0.52)
 Линейные графики созданы



5. СКРИПИЧНАЯ ДИАГРАММА
 Скрипичная диаграмма создана



6. ДИАГРАММА РАЗМАХА (BOX PLOT)
✓ Диаграмма размаха создана



----------------------------------------
АНАЛИЗ ВЫБРОСОВ (метод IQR):
----------------------------------------
 ВЫВОД: Выбросы не обнаружены

7. СТОЛБЧАТАЯ ДИАГРАММА (max разница квартилей)
 Признак с МАКСИМАЛЬНОЙ разницей Q3-Q1: len
 Разница квартилей: 38.07
 Столбчатая диаграмма создана



8. ГИСТОГРАММА (thick vs len)
 Гистограмма создана



9. 3D ЛИНЕЙНЫЙ ГРАФИК
 3D линейный график создан



10. 3D ДИАГРАММА РАССЕЯНИЯ
 3D диаграмма рассеяния создана



СВОДНАЯ СТАТИСТИКА ПО ВСЕМ ПУНКТАМ ЗАДАНИЯ

════════════════════════════════════════════════════════════
 ОСНОВНЫЕ СТАТИСТИКИ ДАННЫХ:
════════════════════════════════════════════════════════════
        thick   width     len
count  475.00  475.00  475.00
mean     2.51   51.01  144.04
std      0.72   12.90   28.59
min      0.80   17.94   72.30
25%      2.00   42.44  125.10
50%      2.51   51.36  143.35
75%      2.99   59.67  163.17
max      4.25   83.32  214.84

════════════════════════════════════════════════════════════
 РАСПРЕДЕЛЕНИЕ ПО КЛАССАМ:
════════════════════════════════════════════════════════════
  Класс 1: 104 записей ( 21.9%)
  Класс 2:  95 записей ( 20.0%)
  Класс 3:  89 записей ( 18.7%)
  Класс 4:  85 записей ( 17.9%)
  Класс 5:  66 записей ( 13.9%)
  Класс n:  36 записей (  7.6%)

════════════════════════════════════════════════════════════
 КОРРЕЛЯЦИОННАЯ МАТРИЦА:
════════════════════════════════════════════════════════════
       thick  width    len
thick  1.000 -0.0